In [1]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [2]:
# define variables
wetgrass_model = BayesianModel([('R', 'N'), 
                                ('R', 'M'),
                                ('S', 'M')])

# define parameters
cpd_R = TabularCPD(variable='R', variable_card=2, values=[[0.9], [0.1]])
cpd_S = TabularCPD(variable='S', variable_card=2, values=[[0.9], [0.1]])
cpd_N = TabularCPD(variable='N', variable_card=2, values=[[0.99, 0.01], [0.01, 0.99]],
                  evidence=['R'], evidence_card=[2])
cpd_M = TabularCPD(variable='M', variable_card=2, values=[[0.99, 0.01, 0.01, 0.01],
                                                          [0.01, 0.99, 0.99, 0.99]],
                   evidence=['R', 'S'], evidence_card=[2, 2])

In [3]:
# Associating the parameters with the model structure.
wetgrass_model.add_cpds(cpd_R, cpd_S, cpd_N, cpd_M)

# Checking if the cpds are valid for the model.
wetgrass_model.check_model()

True

In [4]:
wetgrass_model.nodes()

['R', 'N', 'M', 'S']

In [5]:
wetgrass_model.edges()

[('R', 'N'), ('R', 'M'), ('S', 'M')]

In [6]:
# Doing some simple queries on the network
wetgrass_model.is_active_trail('R', 'S')

False

In [7]:
wetgrass_model.is_active_trail('R', 'S', observed=['M'])

True

In [8]:
wetgrass_model.get_independencies()

(R _|_ S)
(R _|_ S | N)
(N _|_ S)
(N _|_ M, S | R)
(N _|_ S | R, M)
(N _|_ M | R, S)
(M _|_ N | R)
(M _|_ N | R, S)
(S _|_ R, N)
(S _|_ N | R)
(S _|_ R | N)
(S _|_ N | R, M)

In [9]:
# Doing exact inference using Variable Elimination
wetgrass_infer = VariableElimination(wetgrass_model)
m = wetgrass_infer.query(variables=['M'])
print(m['M'])

╒═════╤══════════╕
│ M   │   phi(M) │
╞═════╪══════════╡
│ M_0 │   0.8038 │
├─────┼──────────┤
│ M_1 │   0.1962 │
╘═════╧══════════╛


In [10]:
r = wetgrass_infer.query(variables=['R'], evidence={'M': 1})
print(r['R'])

╒═════╤══════════╕
│ R   │   phi(R) │
╞═════╪══════════╡
│ R_0 │   0.4954 │
├─────┼──────────┤
│ R_1 │   0.5046 │
╘═════╧══════════╛


In [11]:
r = wetgrass_infer.query(variables=['R'], evidence={'M': 1, 'N': 1})
print(r['R'])

╒═════╤══════════╕
│ R   │   phi(R) │
╞═════╪══════════╡
│ R_0 │   0.0098 │
├─────┼──────────┤
│ R_1 │   0.9902 │
╘═════╧══════════╛
